In [10]:
import torch
import torch.nn as nn
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import SGConv
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

## Data

First we load the predefined Planetoid dataset

In [2]:
path = './data'
dataset = Planetoid(path, "Cora")
data = dataset[0]
print('Cora', data)

Processing...


Cora Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


Done!


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device', device)

Device cpu


## Model

Let's construct our graph neural network, whcih will consist of a single SGConv layer only.

SGConv: https://pytorch-geometric.readthedocs.io/en/latest/_modules/torch_geometric/nn/conv/sg_conv.html

In [9]:
# Simple Graph Convolutional Neural Network
model = SGConv(
    in_channels=data.num_features,    # dimension of the input vectors
    out_channels=dataset.num_classes, # dimension of the output vectors
    K=1,                              # size of neighborhood for each node (1 means only look at direct neighbors)
    cached=True                       # cache can save time, but required more memory
)

print("In Channels:", data.num_features)
print("Out Channels:", dataset.num_classes)

In Channels: 1433
Out Channels: 7


### Compute embeddings for nodes

Before we train the model let's just pass some data through it to see how the embeddings look like.

In [8]:
print("Shape of the original data:", data.x.shape)
print("Shape of the embedded data:", model(data.x, data.edge_index).shape)

Shape of the original data: torch.Size([2708, 1433])
Shape of the embedded data: torch.Size([2708, 7])


The input vectors have 1433 components whereas the output vectors only have seven, which corresponds to the number of classes in the dataset.

### More complex model

Now let's build a more complex model that consists of multiple layers. 

In [49]:
class SGNet(nn.Module):
    def __init__(self, input_dim, output_dim, K=1):
        super().__init__()
        self.conv1 = SGConv(
            in_channels=input_dim,
            out_channels=output_dim,
            K=K,
            cached=True
        )
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, data):
        x = self.conv1(data.x, data.edge_index) # Pass the data through the SGConv layer
        x = self.softmax(x)                     # Turns the output into a probability distrobution
        return x

In [50]:
# Instantiate the model and the data and move them to the chosen device (CPU / CUDA)
model, data = SGNet(data.num_features, dataset.num_classes).to(device), data.to(device)

# Select an optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.2, weight_decay=0.005)

#  Loss function
criterion = nn.CrossEntropyLoss()

In [51]:
# Inspect the shape of te parameters
for i, param in model.named_parameters():
    print("Parameter {}:".format(i))
    print("Shape:", param.shape)

Parameter conv1.lin.weight:
Shape: torch.Size([7, 1433])
Parameter conv1.lin.bias:
Shape: torch.Size([7])


The graph convolution layer has two parameter matrices:
- The matrix that is multiplied with the feature vectors of the nodes
- The bias that is added to the results of the convolution operation

## Train the model

In [52]:
def train():
    # Set the model to training mode
    model.train()
    
    # Reset the gradients
    optimizer.zero_grad()
    
    # Forward the data through the model and compute predictions.
    predicted_classes = model(data)
    
    # Compute loss
    loss = criterion(predicted_classes[data.train_mask], data.y[data.train_mask])
    
    # Backpropagation
    loss.backward()
    
    # Update the parameters
    optimizer.step()

In [53]:
def test():
    # Set the model ot evaluation mode
    model.eval()
    
    # Compute logits
    logits = model(data)
    
    # Accuracies for training, validation and test data
    accs = []
    
    # Compute accuracies for training, validation and test dataset
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        # Get predicted class
        pred = logits[mask].max(1)[1]
        #  Compute the accuracy
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        # Save accuracy
        accs.append(acc)
        
    return accs

In [54]:
best_val_acc = test_acc = 0

for epoch in range(1, 100):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    msg = "Epoch: {:03d}, Train: {:.4f}, Val: {:.4f}, Test: {:.4f}"
    print(msg.format(epoch, train_acc, val_acc, test_acc))

Epoch: 001, Train: 0.9500, Val: 0.4680, Test: 0.4910
Epoch: 002, Train: 0.9357, Val: 0.7180, Test: 0.7150
Epoch: 003, Train: 0.9643, Val: 0.7120, Test: 0.7150
Epoch: 004, Train: 0.9643, Val: 0.6300, Test: 0.7150
Epoch: 005, Train: 0.9500, Val: 0.6820, Test: 0.7150
Epoch: 006, Train: 0.9714, Val: 0.7180, Test: 0.7150
Epoch: 007, Train: 0.9714, Val: 0.7360, Test: 0.7850
Epoch: 008, Train: 0.9714, Val: 0.7300, Test: 0.7850
Epoch: 009, Train: 0.9714, Val: 0.7220, Test: 0.7850
Epoch: 010, Train: 0.9714, Val: 0.7040, Test: 0.7850
Epoch: 011, Train: 0.9643, Val: 0.7020, Test: 0.7850
Epoch: 012, Train: 0.9643, Val: 0.7100, Test: 0.7850
Epoch: 013, Train: 0.9500, Val: 0.7080, Test: 0.7850
Epoch: 014, Train: 0.9571, Val: 0.7060, Test: 0.7850
Epoch: 015, Train: 0.9571, Val: 0.7140, Test: 0.7850
Epoch: 016, Train: 0.9643, Val: 0.7100, Test: 0.7850
Epoch: 017, Train: 0.9643, Val: 0.7140, Test: 0.7850
Epoch: 018, Train: 0.9786, Val: 0.7300, Test: 0.7850
Epoch: 019, Train: 0.9786, Val: 0.7400, Test: 

We can see a clear overfitting of teh model. The training set reaches an accuracy of 96%, wherease the validation set achieves 74% and the test set achieves 77% accuracy.